In [149]:
# Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [150]:
# load data
df = pd.read_csv('../../data/H2.csv')

In [151]:
#  Train-Test Split
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_df.shape, test_df.shape

((63464, 31), (15866, 31))

In [152]:
# Handle missing values
train_df['Children'].fillna(0, inplace=True)  # Fill missing 'Children' with 0
train_df['Country'].fillna('Unknown', inplace=True)  # Fill missing 'Country' with 'Unknown'

# Drop rows where adults, babies and children are zero at the same time
train_df = train_df[(train_df['Adults'] != 0) | (train_df['Babies'] != 0) | (train_df['Children'] != 0)] 

In [153]:
# Remove Outliers
from sklearn.ensemble import IsolationForest

# Get only numerical data
train_df_num = train_df.select_dtypes(include=[np.number])
# Get only Categorical data
train_df_cat = train_df.select_dtypes(include=['object'])

# Predict Outliers
isolation_forest = IsolationForest(random_state=42)
outlier_pred = isolation_forest.fit_predict(train_df_num)

# Exclude outliers
train_df = train_df.iloc[outlier_pred == 1]

train_df.shape

(58419, 31)

In [154]:
# Define a dictionary to map country codes to regions
country_to_region = {
    "PRT": "Europe",
    "GBR": "Europe",
    "ESP": "Europe",
    "IRL": "Europe",
    "NLD": "Europe",
    "RUS": "Europe",
    "FRA": "Europe",
    "DEU": "Europe",
    "ITA": "Europe",
    "BEL": "Europe",
    "CHE": "Europe",
    "USA": "North America",
    "CAN": "North America",
    "MEX": "North America",
    "CN": "Asia",
    "BRA": "South America",
    "ARG": "South America",
    "AUS": "Oceania",
    "Unknown": "Unknown",
}

# Map the country codes to regions
train_df["Region"] = train_df["Country"].map(country_to_region)

# Drop the Country column
train_df.drop(columns=["Country"], inplace=True)

In [155]:
# Remove blank spaces from the 'Agent' column
train_df['Agent'] = train_df['Agent'].str.strip()

# Replace 'NULL' values in the 'Agent' column with 0
train_df['Agent'].replace('NULL', 0, inplace=True)

# Convert the 'Agent' column to integer
train_df['Agent'] = train_df['Agent'].astype(int)

In [156]:
# Columns to drop from the dataset
columns_to_drop = [
    'IsCanceled', 
    'ReservationStatus', 
    'ReservationStatusDate', 
    'ArrivalDateYear', 
    'ArrivalDateMonth', 
    'ArrivalDateWeekNumber', 
    'ArrivalDateDayOfMonth', 
    'Company'
]

# Drop the specified columns
train_df = train_df.drop(columns=columns_to_drop)

##### Create Data Preprocessing Pipeline for **Customer Segmentation** Application 

In [157]:
# Create new Features

# TotalStay
train_df["TotalStay"] = train_df["StaysInWeekendNights"] + train_df["StaysInWeekNights"]

# SpendingPerPerson
train_df["SpendingPerPerson"] = train_df["ADR"] / (train_df["Adults"] + train_df["Children"] + train_df["Babies"])
train_df["SpendingPerPerson"].fillna(0, inplace=True)  # Handle division by zero

# GroupType
def determine_group_type(row):
    if row["Adults"] == 1 and (row["Children"] + row["Babies"] == 0):
        return "Single"
    elif row["Adults"] == 2 and (row["Children"] + row["Babies"] == 0):
        return "Couple"
    elif (row["Children"] + row["Babies"]) > 0:
        return "Family"
    elif row["Adults"] > 2:
        return "Big_Group"
    else:
        return "Other"

train_df["GroupType"] = train_df.apply(determine_group_type, axis=1)

# BookingUrgency
def determine_booking_urgency(lead_time):
    if lead_time > 60:
        return "Early_Booking"
    elif 30 <= lead_time <= 60:
        return "Medium_Term_Booking"
    else:
        return "Last_Minute_Booking"

train_df["BookingUrgency"] = train_df["LeadTime"].apply(determine_booking_urgency)

# HighDemand
train_df["HighDemand"] = train_df["TotalOfSpecialRequests"].apply(lambda x: "High" if x >= 2 else "Low")

# LoyaltyScore
train_df["LoyaltyScore"] = train_df["IsRepeatedGuest"] * (train_df["PreviousBookingsNotCanceled"] + 1)

# ChildRatio
train_df["ChildRatio"] = ((train_df["Children"] + train_df["Babies"]) / (train_df["Adults"] + train_df["Children"] + train_df["Babies"])).round(2)
train_df["ChildRatio"].fillna(0, inplace=True)  # Handle division by zero

# Occupancy
train_df["Occupancy"] = np.where(train_df["TotalStay"] == 0, 0, 
                                (train_df["Adults"] + train_df["Children"] + train_df["Babies"]) / train_df["TotalStay"]).round(2)

# WaitingTimeCategory
def determine_waiting_time_category(days):
    if days < 7:
        return "Short"
    elif 7 <= days <= 30:
        return "Medium"
    else:
        return "Long"

train_df["WaitingTimeCategory"] = train_df["DaysInWaitingList"].apply(determine_waiting_time_category)

# CancellationRisk
train_df["CancellationRisk"] = train_df["PreviousCancellations"] * train_df["LeadTime"]

# Check new Features
train_df.head()

,LeadTime,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,Meal,MarketSegment,DistributionChannel,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,ReservedRoomType,AssignedRoomType,BookingChanges,DepositType,Agent,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,Region,TotalStay,SpendingPerPerson,GroupType,BookingUrgency,HighDemand,LoyaltyScore,ChildRatio,Occupancy,WaitingTimeCategory,CancellationRisk
12826,71,0,3,1,0.0,0,BB,Offline TA/TO,TA/TO,0,0,0,A,A,0,Non Refund,236,0,Transient,120.00,0,0,Europe,3,120.00,Single,Early_Booking,Low,0,0.00,0.33,Short,0
36957,0,0,2,2,0.0,0,BB,Direct,Direct,0,0,0,A,A,0,No Deposit,0,0,Transient,106.00,0,1,Europe,2,53.00,Couple,Last_Minute_Booking,Low,0,0.00,1.00,Short,0
46297,0,0,1,2,1.0,0,BB,Online TA,TA/TO,0,0,0,A,D,0,No Deposit,9,0,Transient,138.00,0,0,Europe,1,46.00,Family,Last_Minute_Booking,Low,0,0.33,3.00,Short,0
12562,27,0,1,2,0.0,0,BB,Offline TA/TO,Corporate,0,0,0,A,A,0,No Deposit,191,0,Transient-Party,120.00,0,0,Europe,1,60.00,Couple,Last_Minute_Booking,Low,0,0.00,2.00,Short,0
37239,256,1,2,1,0.0,0,BB,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,4,0,Transient-Party,64.33,0,0,South America,3,64.33,Single,Early_Booking,Low,0,0.00,0.33,Short,0


In [158]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Numeric and Categorical Columns
numeric_columns = train_df.select_dtypes(include=[np.number]).columns
categorical_columns = train_df.select_dtypes(include=["object"]).columns

numeric_columns, categorical_columns


(Index(['LeadTime', 'StaysInWeekendNights', 'StaysInWeekNights', 'Adults',
        'Children', 'Babies', 'IsRepeatedGuest', 'PreviousCancellations',
        'PreviousBookingsNotCanceled', 'BookingChanges', 'Agent',
        'DaysInWaitingList', 'ADR', 'RequiredCarParkingSpaces',
        'TotalOfSpecialRequests', 'TotalStay', 'SpendingPerPerson',
        'LoyaltyScore', 'ChildRatio', 'Occupancy', 'CancellationRisk'],
       dtype='object'),
 Index(['Meal', 'MarketSegment', 'DistributionChannel', 'ReservedRoomType',
        'AssignedRoomType', 'DepositType', 'CustomerType', 'Region',
        'GroupType', 'BookingUrgency', 'HighDemand', 'WaitingTimeCategory'],
       dtype='object'))

In [159]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

# Pipeline for numeric columns
numeric_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

# Pipeline for categorical columns
categorical_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

# Define processors for both types
preprocessor = ColumnTransformer([
    ("num", numeric_pipeline, numeric_columns),
    ("cat", categorical_pipeline, categorical_columns)
])

In [160]:
# Transform data and put the result into a DataFrame
processed_data = preprocessor.fit_transform(train_df)

# Let's get the column names to convert the resulting data to DataFrame format
# Get categorical column names from OneHotEncoder
categorical_feature_names = preprocessor.named_transformers_["cat"]["onehot"].get_feature_names_out(categorical_columns)

# Concatenate new column names
all_feature_names = list(numeric_columns) + list(categorical_feature_names)

# Convert transformed data to DataFrame
processed_df = pd.DataFrame(processed_data, columns=all_feature_names)

# Let's check the first few rows of processed data
processed_df.head()

,LeadTime,StaysInWeekendNights,StaysInWeekNights,Adults,Children,Babies,IsRepeatedGuest,PreviousCancellations,PreviousBookingsNotCanceled,BookingChanges,Agent,DaysInWaitingList,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,TotalStay,SpendingPerPerson,LoyaltyScore,ChildRatio,Occupancy,CancellationRisk,Meal_BB,Meal_FB,Meal_HB,Meal_SC,MarketSegment_Aviation,MarketSegment_Complementary,MarketSegment_Corporate,MarketSegment_Direct,MarketSegment_Groups,MarketSegment_Offline TA/TO,MarketSegment_Online TA,MarketSegment_Undefined,DistributionChannel_Corporate,DistributionChannel_Direct,DistributionChannel_GDS,DistributionChannel_TA/TO,DistributionChannel_Undefined,ReservedRoomType_A,ReservedRoomType_B,ReservedRoomType_C,ReservedRoomType_D,ReservedRoomType_E,ReservedRoomType_F,ReservedRoomType_G,AssignedRoomType_A,AssignedRoomType_B,AssignedRoomType_C,AssignedRoomType_D,AssignedRoomType_E,AssignedRoomType_F,AssignedRoomType_G,AssignedRoomType_K,DepositType_No Deposit,DepositType_Non Refund,DepositType_Refundable,CustomerType_Contract,CustomerType_Group,CustomerType_Transient,CustomerType_Transient-Party,Region_Asia,Region_Europe,Region_North America,Region_Oceania,Region_South America,Region_Unknown,GroupType_Big_Group,GroupType_Couple,GroupType_Family,GroupType_Single,BookingUrgency_Early_Booking,BookingUrgency_Last_Minute_Booking,BookingUrgency_Medium_Term_Booking,HighDemand_High,HighDemand_Low,WaitingTimeCategory_Long,WaitingTimeCategory_Medium,WaitingTimeCategory_Short
0,-0.361918,-0.928602,0.700687,-1.801339,-0.198501,-0.052421,-0.056363,-0.237691,-0.026791,-0.305665,3.750463,-0.159691,0.373619,-0.099877,-0.691836,0.062646,2.105765,-0.053025,-0.210675,-0.955101,-0.210687,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,-1.004663,-0.928602,-0.110953,0.274292,-0.198501,-0.052421,-0.056363,-0.237691,-0.026791,-0.305665,-0.473984,-0.159691,0.037733,-0.099877,0.637387,-0.598908,-0.192796,-0.053025,-0.210675,0.272689,-0.210687,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,-1.004663,-0.928602,-0.922593,0.274292,3.611044,-0.052421,-0.056363,-0.237691,-0.026791,-0.305665,-0.312882,-0.159691,0.805471,-0.099877,-0.691836,-1.260462,-0.432944,-0.053025,3.874221,3.937736,-0.210687,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
3,-0.760239,-0.928602,-0.922593,0.274292,-0.198501,-0.052421,-0.056363,-0.237691,-0.026791,-0.305665,2.944954,-0.159691,0.373619,-0.099877,-0.691836,-1.260462,0.047353,-0.053025,-0.210675,2.105212,-0.210687,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
4,1.312840,0.279567,-0.110953,-1.801339,-0.198501,-0.052421,-0.056363,-0.237691,-0.026791,-0.305665,-0.402383,-0.159691,-0.962006,-0.099877,-0.691836,0.062646,0.195901,-0.053025,-0.210675,-0.955101,-0.210687,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [161]:
processed_df.shape

(58419, 78)